In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

#1. Veri Setini Yükle
file_path = "sample_data/ds_salaries.csv"
df = pd.read_csv(file_path)

#2. Kullanılacak Sütunları Seç
features = ["work_year", "experience_level", "employment_type", "job_title",
            "salary_in_usd", "employee_residence", "remote_ratio", "company_location"]
target = "company_size"

#3. Eksik Verileri Temizle
df = df.dropna()

#4. Hedef Değişkeni Sayısal Hale Getir (S, M, L → 0, 1, 2)
label_encoder = LabelEncoder()
df[target] = label_encoder.fit_transform(df[target])

#5. Kategorik Değişkenleri One-Hot Encoding ile Sayısallaştır
df_encoded = pd.get_dummies(df[features])

#6. Veriyi Eğitim ve Test Setlerine Ayır
X_train, X_test, y_train, y_test = train_test_split(df_encoded, df[target], test_size=0.2, random_state=42)

#7. Modeli Eğit
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

#8. Modelin Tahminlerini Al
y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score

#8. Modelin Tahminlerini Al
y_pred = model.predict(X_test)

#9. Performans Metriklerini Hesapla
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Veri yükleme ve ön işleme tamamlandı!")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Skoru: {f1:.4f}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#7. Modeli Eğit (Random Forest)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("Model eğitildi!")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize

# 1. Modelin Doğruluk Oranını Ölç
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluk Oranı: {accuracy:.2%}")

#2. Confusion Matrix Görselleştir
plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Tahmin Edilen")
plt.ylabel("Gerçek Değer")
plt.title("Confusion Matrix - Şirket Büyüklüğü Tahmini")
plt.show()

#3. Sınıf Başarı Durumu (Classification Report)
print(" Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

#4. ROC Curve ve AUC Skoru (Çoklu Sınıf Durumu İçin)
y_test_bin = label_binarize(y_test, classes=[0, 1, 2])  # 3 sınıf olduğu varsayılıyor
y_pred_prob = model.predict_proba(X_test)

# ROC eğrisini her sınıf için çiz
plt.figure(figsize=(10, 7))
for i in range(3):  # 3 sınıf
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_pred_prob[:, i])
    auc = roc_auc_score(y_test_bin[:, i], y_pred_prob[:, i])
    plt.plot(fpr, tpr, label=f"Sınıf {label_encoder.classes_[i]} (AUC = {auc:.2f})")

plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="best")
plt.show()

#5. Özellik Önem Skoru Görselleştir
importances = model.feature_importances_
feature_names = X_train.columns

sorted_idx = importances.argsort()[-10:]
plt.figure(figsize=(8, 6))
plt.barh([feature_names[i] for i in sorted_idx], importances[sorted_idx], color="skyblue")
plt.xlabel("Önem Skoru")
plt.title("En Önemli 10 Özellik")
plt.show()

#6. Sınıfların Dağılımı (Sınıf Dağılımı)
plt.figure(figsize=(8, 5))
sns.countplot(x=y_test, palette="Set2")
plt.title("Gerçek Değer Sınıf Dağılımı")
plt.xlabel("Şirket Büyüklüğü")
plt.ylabel("Frekans")
plt.show()